In [1]:
import json
import pymysql
import pandas as pd
from tqdm import tqdm

In [2]:
con = pymysql.connect(
    host='0.0.0.0',
    port=55000,
    user='root',
    password='mysqlpw',
    database='ddb'
)

In [3]:

VAR = []

for table in ["D_S", "D_M"]:
    SQL = f"""
    SHOW FULL COLUMNS FROM {table}
    """

    cols = pd.read_sql(SQL, con)
    cols = [icol for icol in cols["Field"] if "@" in icol]

    for icol in tqdm(cols[:]):
        SQL = f"""
        SELECT
            TA.StationID,
            TA.X AS X,
            TB.T AS Y
        FROM
        (
            SELECT
                StationID, STD(`{icol}`) AS X
            FROM {table}
            GROUP BY StationID
        ) AS TA
        LEFT JOIN (
            SELECT
                StationID, 2 * ASIN(
                    SQRT(
                        SIN((Latitude-31.89)/2) * SIN((Latitude-31.89)/2) + 
                        COS(Latitude) * COS(31.89) * SIN((Longitude-104.46)/2) * SIN((Longitude-104.46)/2)
                    )
                ) * 6371 AS T
            FROM D_SInfo
            ORDER BY T
        ) AS TB
        ON TA.StationID = TB.StationID
        """

        data = pd.read_sql(SQL, con)
        VAR.append([icol, abs(data.corr()["X"]["Y"])])


100%|██████████| 51/51 [00:16<00:00,  3.15it/s]


In [4]:
pd.DataFrame(VAR, columns=["VAR", "CORR"]).sort_values("CORR", ascending=False)

,VAR,CORR
47,magn@kurt,0.198905
46,magn@skew,0.197935
89,magn@ulf_kurt,0.195142
73,magn@level5d_abs_mean,0.167132
62,magn@power_40_60,0.162849
...,...,...
82,magn@level6d_energy,0.004320
90,magn@ulf_abs_max,0.003736
60,magn@power_30_35,0.002270
85,magn@ulf_abs_mean,0.000938
